## Char-RNN

In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import tensorflow.experimental.numpy as tnp
tf.enable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
lyrics = open("Lyrics_Counterparts.txt", encoding="utf-8").read().lower()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lyrics)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[17, 5, 8, 9, 3]]

In [5]:
max_id = len(tokenizer.word_index) #number of distinct characters in text
dataset_size = tokenizer.document_count #total number of characters
print(max_id, " ", dataset_size)

52   71707


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([lyrics])) - 1
#encodes the full text so each character is represented by its ID

In [7]:
#splitting dataset for training/test/validation
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
#convert long string of characters into windows for training(batches)
n_steps = 100
window_length = n_steps + 1 #target character = input shifted 1 char ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
#flatten the dataset for training
dataset = dataset.flat_map(lambda window: window.batch(window_length))
#this gives us a single tensor for each window

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
dataset = dataset.prefetch(1)

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
2014/2014 [==============================] - 2115s 1s/step - loss: 2.2474
Epoch 2/10
2014/2014 [==============================] - 2157s 1s/step - loss: 1.4097
Epoch 3/10
2014/2014 [==============================] - 2119s 1s/step - loss: 1.2683
Epoch 4/10
2014/2014 [==============================] - 2118s 1s/step - loss: 1.2004
Epoch 5/10
2014/2014 [==============================] - 2119s 1s/step - loss: 1.1599
Epoch 6/10
2014/2014 [==============================] - 2114s 1s/step - loss: 1.1359
Epoch 7/10
2014/2014 [==============================] - 2118s 1s/step - loss: 1.1140
Epoch 8/10
2014/2014 [==============================] - 2112s 1s/step - loss: 1.0982
Epoch 9/10
2014/2014 [==============================] - 2128s 1s/step - loss: 1.0834
Epoch 10/10
2014/2014 [==============================] - 2115s 1s/step - loss: 1.0732


In [14]:
model.save('char_rnn_lyrics')

In [15]:
tf.executing_eagerly()

True

In [16]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [17]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new,steps=1)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/ temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
#this function picks the next character randomly, with a probability equal to the estimated probability,using tf.random.categorical
#categorical() samples random class indices, given the class probs (logits)
#temperature is a variables that controls the diversity of the characters generated
# a near-zero temperature will output characters with high probabilities
# a high temperature will give all characters equal probability

In [18]:
#this function repeatedly calls the above function to generate text
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char (text, temperature)
    return text

In [19]:
print(complete_text("m",temperature=1)+ complete_text("i",temperature=1)+ complete_text("t",temperature=1)+complete_text("c",temperature=1)+complete_text("h",temperature=1)) 

assed my pasts
as leaves you could feel safer in myself from sinking
whilh a world creater to sink
yo


In [20]:
print(complete_text("b", temperature=1))

bathed would hoped is god who i am
and we realize that change everything, i can never asked they
resu


In [21]:
print(complete_text("f", temperature=1))

felfs than god from the right will be remembered
we will be remembered
we've let you to what i won’t 


## Stateful RNN

In [22]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=n_steps, drop_remainder=True)

In [23]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [24]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [25]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.3, recurrent_dropout=0.3, batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])

In [26]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [27]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps

In [28]:
history = model.fit(dataset, epochs=500, callbacks=[ResetStatesCallback()], steps_per_epoch=steps_per_epoch)

Epoch 1/500
20/20 [==============================] - 23s 906ms/step - loss: 3.6693
Epoch 2/500
20/20 [==============================] - 18s 892ms/step - loss: 3.0281
Epoch 3/500
20/20 [==============================] - 18s 899ms/step - loss: 3.0027
Epoch 4/500
20/20 [==============================] - 19s 928ms/step - loss: 2.9981
Epoch 5/500
20/20 [==============================] - 19s 938ms/step - loss: 2.9964
Epoch 6/500
20/20 [==============================] - 18s 923ms/step - loss: 2.9951
Epoch 7/500
20/20 [==============================] - 19s 934ms/step - loss: 2.9840
Epoch 8/500
20/20 [==============================] - 19s 934ms/step - loss: 2.9351
Epoch 9/500
20/20 [==============================] - 19s 936ms/step - loss: 2.8763
Epoch 10/500
20/20 [==============================] - 18s 922ms/step - loss: 2.8087
Epoch 11/500
20/20 [==============================] - 19s 942ms/step - loss: 2.7455
Epoch 12/500
20/20 [==============================] - 19s 932ms/step - loss: 2.6929
E

20/20 [==============================] - 19s 925ms/step - loss: 1.6746
Epoch 99/500
20/20 [==============================] - 19s 926ms/step - loss: 1.6741
Epoch 100/500
20/20 [==============================] - 19s 930ms/step - loss: 1.6659
Epoch 101/500
20/20 [==============================] - 18s 924ms/step - loss: 1.6670
Epoch 102/500
20/20 [==============================] - 19s 933ms/step - loss: 1.6591
Epoch 103/500
20/20 [==============================] - 19s 936ms/step - loss: 1.6599
Epoch 104/500
20/20 [==============================] - 19s 930ms/step - loss: 1.6513
Epoch 105/500
20/20 [==============================] - 19s 947ms/step - loss: 1.6600
Epoch 106/500
20/20 [==============================] - 19s 933ms/step - loss: 1.6613
Epoch 107/500
20/20 [==============================] - 19s 936ms/step - loss: 1.6512
Epoch 108/500
20/20 [==============================] - 18s 917ms/step - loss: 1.6419
Epoch 109/500
20/20 [==============================] - 19s 928ms/step - loss: 1.

20/20 [==============================] - 19s 945ms/step - loss: 1.5010
Epoch 197/500
20/20 [==============================] - 19s 931ms/step - loss: 1.5137
Epoch 198/500
20/20 [==============================] - 19s 941ms/step - loss: 1.4955
Epoch 199/500
20/20 [==============================] - 18s 924ms/step - loss: 1.5098
Epoch 200/500
20/20 [==============================] - 19s 942ms/step - loss: 1.5059
Epoch 201/500
20/20 [==============================] - 19s 935ms/step - loss: 1.4967
Epoch 202/500
20/20 [==============================] - 19s 928ms/step - loss: 1.5107
Epoch 203/500
20/20 [==============================] - 19s 929ms/step - loss: 1.5051
Epoch 204/500
20/20 [==============================] - 19s 944ms/step - loss: 1.4976
Epoch 205/500
20/20 [==============================] - 19s 936ms/step - loss: 1.5003
Epoch 206/500
20/20 [==============================] - 18s 924ms/step - loss: 1.5080
Epoch 207/500
20/20 [==============================] - 19s 930ms/step - loss: 1

20/20 [==============================] - 19s 930ms/step - loss: 1.4466
Epoch 293/500
20/20 [==============================] - 19s 930ms/step - loss: 1.4413
Epoch 294/500
20/20 [==============================] - 18s 924ms/step - loss: 1.4480
Epoch 295/500
20/20 [==============================] - 19s 928ms/step - loss: 1.4403
Epoch 296/500
20/20 [==============================] - 19s 928ms/step - loss: 1.4408
Epoch 297/500
20/20 [==============================] - 18s 922ms/step - loss: 1.4447
Epoch 298/500
20/20 [==============================] - 19s 939ms/step - loss: 1.4397
Epoch 299/500
20/20 [==============================] - 19s 936ms/step - loss: 1.4415
Epoch 300/500
20/20 [==============================] - 19s 931ms/step - loss: 1.4388
Epoch 301/500
20/20 [==============================] - 19s 932ms/step - loss: 1.4450
Epoch 302/500
20/20 [==============================] - 19s 938ms/step - loss: 1.4451
Epoch 303/500
20/20 [==============================] - 19s 960ms/step - loss: 1

20/20 [==============================] - 19s 926ms/step - loss: 1.4098
Epoch 389/500
20/20 [==============================] - 18s 921ms/step - loss: 1.4020
Epoch 390/500
20/20 [==============================] - 19s 937ms/step - loss: 1.4069
Epoch 391/500
20/20 [==============================] - 19s 934ms/step - loss: 1.4128
Epoch 392/500
20/20 [==============================] - 18s 921ms/step - loss: 1.4087
Epoch 393/500
20/20 [==============================] - 19s 935ms/step - loss: 1.4051
Epoch 394/500
20/20 [==============================] - 19s 933ms/step - loss: 1.4026
Epoch 395/500
20/20 [==============================] - 19s 930ms/step - loss: 1.4072
Epoch 396/500
20/20 [==============================] - 19s 929ms/step - loss: 1.4102
Epoch 397/500
20/20 [==============================] - 19s 928ms/step - loss: 1.4033
Epoch 398/500
20/20 [==============================] - 19s 929ms/step - loss: 1.4083
Epoch 399/500
20/20 [==============================] - 19s 924ms/step - loss: 1

20/20 [==============================] - 18s 923ms/step - loss: 1.3891
Epoch 485/500
20/20 [==============================] - 19s 930ms/step - loss: 1.3740
Epoch 486/500
20/20 [==============================] - 19s 928ms/step - loss: 1.3846
Epoch 487/500
20/20 [==============================] - 19s 931ms/step - loss: 1.3870
Epoch 488/500
20/20 [==============================] - 18s 921ms/step - loss: 1.3921
Epoch 489/500
20/20 [==============================] - 19s 924ms/step - loss: 1.3809
Epoch 490/500
20/20 [==============================] - 19s 925ms/step - loss: 1.3851
Epoch 491/500
20/20 [==============================] - 18s 920ms/step - loss: 1.3737
Epoch 492/500
20/20 [==============================] - 19s 944ms/step - loss: 1.3873
Epoch 493/500
20/20 [==============================] - 19s 950ms/step - loss: 1.3808
Epoch 494/500
20/20 [==============================] - 19s 934ms/step - loss: 1.3801
Epoch 495/500
20/20 [==============================] - 18s 922ms/step - loss: 1

In [29]:
model.save('stateful_rnn_lyrics')

In [30]:
tf.executing_eagerly()

True

In [31]:
#stateless copy to allow us to use different batch sizes
stateless_model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [32]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [33]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [34]:
print(complete_text("t"))

tards will saved to fall in a drough gaterage
we feel sold resmleed the weight like to feel through t


In [36]:
print(complete_text("m")+ complete_text("i")+ complete_text("t")+complete_text("c")+complete_text("h")) 

mever life from the mad on mying day from a god
i’m no longer from shallow somewhith your absence to id back and i have leaved grow you live
(home all i hate to sleep
every thing world i could
nothing ltrad your imprace
allow me a memory to empty imprace to we can grow it might had
remembears away to lchor with yourself?
so we know in the dissoven us draging again

[intro]
reserve ad no pain to be warhewrace it that can’t one, the defisning in my pection
but you've’ weres i know what you every being 
